In [1]:
import os
import pandas as pd
from Data_loader import Dataset_Custom
import argparse
import warnings
from tools import EarlyStopping
from torch.utils.data import Dataset, DataLoader
from utils import get_data
from Model import ANN
from torch import nn
import torch
import copy
from tqdm import tqdm
import random 
import numpy as np
import torch.nn.functional as F
warnings.filterwarnings("ignore")

In [2]:
parser_train = argparse.ArgumentParser(description='FL')
parser_train.add_argument('--root_path', type=str, default='../Data/GFC12/')
parser_train.add_argument('--dataset_paths', type=list, default=["wf1", "wf2", "wf3", "wf4", "wf5", "wf6", "wf7"])
parser_train.add_argument('--number_clients', type=int, default=7)
parser_train.add_argument('--seq_len', type=int, default=24*4)
parser_train.add_argument('--pred_len', type=int, default=24)
parser_train.add_argument('--label_len', type=int, default=0)
parser_train.add_argument('--train_length', type=int, default=16800)
parser_train.add_argument('--target', type=str, default='target')
parser_train.add_argument('--scale', type=bool, default=True)
parser_train.add_argument('--inverse', type=bool, default=True)
parser_train.add_argument('--k', type=int, default=2)

parser_train.add_argument('--lr', type=float, default=1e-4)
parser_train.add_argument('--fine_tune_lr', type=float, default=1e-5)
parser_train.add_argument('--global_epochs', type=int, default=200)
parser_train.add_argument('--local_epochs', type=int, default=1)
parser_train.add_argument('--fine_tune_epochs', type=int, default=20)
parser_train.add_argument('--patience', type=int, default=3)
parser_train.add_argument('--fed_patience', type=int, default=3)
parser_train.add_argument('--hidden_layers', type=list, default=[64,64,64])
parser_train.add_argument('--input_size', type=int, default=293)
parser_train.add_argument('--output_size', type=int, default=9)


parser_train.add_argument('--warm_up_epochs', type=int, default=15)
parser_train.add_argument('--selection_epochs', type=int, default=4)
parser_train.add_argument('--importance', type=float, default=0)
parser_train.add_argument('--batch_size', type=int, default=64)
parser_train.add_argument('--device', type=str, default='cuda:1' if torch.cuda.is_available() else 'cpu')
parser_train.add_argument('--forecasting_mode', type=str, default='prob')
parser_train.add_argument('--model_type', type=str, default='NN')
parser_train.add_argument('--model_save_path', type=str, default='../Model1/', help='location of model checkpoints')
parser_train.add_argument('--quantiles', type=list, default=[0.1,0.2,0.3,0.4, 0.5,0.6,0.7,0.8, 0.9])
parser_train.add_argument('--decay', type=float, default=0.75)
args_train = parser_train.parse_args(args=[])

In [3]:
from Server import  Server_mul,Server
from Clients import Client_mul,Client
from Train import Trainer,EWCLoss,evaluation,PinballLoss
from utils import get_data, plot_prob_result,seed_everything

In [4]:
seed_everything(1)

In [5]:
clients=[]
for path in tqdm(args_train.dataset_paths):
    args_temp=copy.deepcopy(args_train)
    args_temp=copy.deepcopy(args_train)
    args_temp.dataset_paths=path
    clients.append(Client_mul(args_temp))

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:05<00:00,  1.35it/s]


In [6]:
server = Server_mul(args_train,clients)

In [8]:
server.fed_train()

test performance: [0.14043924942204397, 0.1415245346724987, 0.16793207857400588, 0.1541354089260918, 0.1464996447352922, 0.15679135104666833, 0.15634336425884537]
False [False, False]
ewc: False
wf1 model: 0
Epoch: 0 | Loss: 0.0783
wf1 model: 1
Epoch: 0 | Loss: 0.0759
ewc: False
wf2 model: 0
Epoch: 0 | Loss: 0.0726
wf2 model: 1
Epoch: 0 | Loss: 0.0935
ewc: False
wf3 model: 0
Epoch: 0 | Loss: 0.0870
wf3 model: 1
Epoch: 0 | Loss: 0.1043
ewc: False
wf4 model: 0
Epoch: 0 | Loss: 0.1002
wf4 model: 1
Epoch: 0 | Loss: 0.0929
ewc: False
wf5 model: 0
Epoch: 0 | Loss: 0.0899
wf5 model: 1
Epoch: 0 | Loss: 0.0876
ewc: False
wf6 model: 0
Epoch: 0 | Loss: 0.0725
wf6 model: 1
Epoch: 0 | Loss: 0.0803
ewc: False
wf7 model: 0
Epoch: 0 | Loss: 0.0995
wf7 model: 1
Epoch: 0 | Loss: 0.1086
Client 0 test loss: [0.0784179724331172, 0.07796575241493729]
Client 1 test loss: [0.08136258780393961, 0.08116580335036763]
Client 2 test loss: [0.10091405151025304, 0.10098357554876579]
Client 3 test loss: [0.0889714147

[ANN(
   (hidden_layers): ModuleList(
     (0): Linear(in_features=293, out_features=64, bias=True)
     (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
   )
   (relu): ReLU()
   (output): Linear(in_features=64, out_features=9, bias=True)
 ),
 ANN(
   (hidden_layers): ModuleList(
     (0): Linear(in_features=293, out_features=64, bias=True)
     (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
   )
   (relu): ReLU()
   (output): Linear(in_features=64, out_features=9, bias=True)
 )]

In [9]:
fed_local_losses=[]
fed_local_preds=[]
fed_local_models=[]
for i in range(args_train.number_clients):
    fed_local_pred,fed_local_loss,fed_local_model=clients[i].fed_local_evaluation()
    fed_local_losses.append(fed_local_loss[server.index_set[i]])
    fed_local_preds.append(fed_local_pred[server.index_set[i]])
    fed_local_models.append(fed_local_model[server.index_set[i]])
print(fed_local_losses)
print(np.mean(fed_local_losses))

[0.07223644668925298, 0.08466056167875251, 0.09076731196566395, 0.08110292202295506, 0.08223103012328278, 0.0757141739053473, 0.09013798964299159]
0.0824072051468923


In [17]:
local_fine_tune_losses=[]
local_fine_tune_preds=[]
local_fine_tune_models=[]
seed_everything(0)
for i in range(args_train.number_clients):
    local_fine_tune_pred,local_fine_tune_loss,local_fine_tune_model=clients[i].local_fine_tune(server.index_set[i],ewc_flag=False,importance=0.02,fine_tune_epochs=20)
    local_fine_tune_losses.append(local_fine_tune_loss)
    local_fine_tune_preds.append(local_fine_tune_pred)
    local_fine_tune_models.append(local_fine_tune_model)
print(local_fine_tune_losses)

1e-05 0
Epoch [1/20] Train Loss: 0.0567 Val Loss: 0.0577
Epoch [2/20] Train Loss: 0.0558 Val Loss: 0.0571
Epoch [3/20] Train Loss: 0.0553 Val Loss: 0.0567
Epoch [4/20] Train Loss: 0.0550 Val Loss: 0.0565
Epoch [5/20] Train Loss: 0.0548 Val Loss: 0.0564
Epoch [6/20] Train Loss: 0.0547 Val Loss: 0.0564
Epoch [7/20] Train Loss: 0.0545 Val Loss: 0.0562
Epoch [8/20] Train Loss: 0.0544 Val Loss: 0.0560
Epoch [9/20] Train Loss: 0.0542 Val Loss: 0.0560
Epoch [10/20] Train Loss: 0.0541 Val Loss: 0.0561
Epoch [11/20] Train Loss: 0.0540 Val Loss: 0.0559
Epoch [12/20] Train Loss: 0.0539 Val Loss: 0.0557
Epoch [13/20] Train Loss: 0.0538 Val Loss: 0.0557
Epoch [14/20] Train Loss: 0.0538 Val Loss: 0.0556
Epoch [15/20] Train Loss: 0.0537 Val Loss: 0.0555
Epoch [16/20] Train Loss: 0.0536 Val Loss: 0.0555
Epoch [17/20] Train Loss: 0.0535 Val Loss: 0.0554
Epoch [18/20] Train Loss: 0.0534 Val Loss: 0.0553
Epoch [19/20] Train Loss: 0.0533 Val Loss: 0.0552
Epoch [20/20] Train Loss: 0.0532 Val Loss: 0.0553
1

In [18]:
fed_local_losses=[]
fed_local_preds=[]
fed_local_models=[]
for i in range(args_train.number_clients):
    fed_local_pred,fed_local_loss,fed_local_model=clients[i].fed_local_evaluation()
    fed_local_losses.append(fed_local_loss[server.index_set[i]])
    fed_local_preds.append(fed_local_pred[server.index_set[i]])
    fed_local_models.append(fed_local_model[server.index_set[i]])
print(fed_local_losses)

[0.07223644668925298, 0.08466056167875251, 0.09076731196566395, 0.08110292202295506, 0.08223103012328278, 0.0757141739053473, 0.09013798964299159]


In [10]:
import pickle

# Save the server object
with open('../result/24/server.pkl', 'wb') as f:
    pickle.dump(server, f)

# Save the clients object
with open('../result/24/clients.pkl', 'wb') as f:
    pickle.dump(clients, f)